## Model Creation on Chunks

In [1]:
train_len = 295246830
one_len = 1589906
zero_len = 293656924
protein_map = {'BRD4': 1, 'HSA': 2, 'sEH': 3}
vocab = {'C': 6825082866, '#': 81527490, '@': 511451694, 'H': 456489972, '=': 1406606874, 'O': 2554179786,
         'N': 2469595230, 'c': 12257477022, '-': 438483636, '.': 216945504, 'l': 491088828, 'B': 123330132,
         'r': 121915914, 'n': 1997759694, 'D': 295246830, 'y': 295246830, 'o': 67918650, 's': 156618468,
         'S': 90662574, 'F': 492710238, '+': 65206260, 'i': 1414026, '/': 11547096, 'I': 23972994}

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.types import LongType, IntegerType, StructType, StructField

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from xgboost.spark import SparkXGBClassifier

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score

In [2]:
import pyspark
pyspark.__version__

'3.5.3'

In [3]:
# for 256 Gb and 64 Cores
spark = (
    SparkSession
    .builder
    .appName("leash belka3")
    .config("spark.driver.memory", "48g")  # Increased driver memory
    .config("spark.executor.memory", "48g")  # Increased executor memory
    .config("spark.executor.instances", "16")  # 16 executors
    .config("spark.executor.cores", "4")  # 4 cores per executor
    .config("spark.driver.maxResultSize", "4g")  # Driver result size limit
    # .config("spark.local.dir", "temp")  # Specify a directory with enough space
    .config("spark.local.dir", "/scratch/23m1521/temp")  # Specify a directory with enough space
    .config("spark.shuffle.file.buffer", "128k")  # Shuffle buffer size
    .config("spark.memory.fraction", "0.8")  # Memory fraction for tasks
    .config("spark.shuffle.memoryFraction", "0.6")  # Shuffle memory fraction
    .config("spark.executor.javaOptions", "-Xmx48g")  # JVM heap size for executors
    .master("local[64]")  # Use all 64 cores on the machine
    .getOrCreate()
)

spark

# SynapseML 
# spark = (
#     SparkSession
#     .builder
#     .appName("leash belka3")
#     .config("spark.driver.memory", "48g")  # Increased driver memory
#     .config("spark.executor.memory", "48g")  # Increased executor memory
#     .config("spark.executor.instances", "16")  # 16 executors
#     .config("spark.executor.cores", "4")  # 4 cores per executor
#     .config("spark.driver.maxResultSize", "4g")  # Driver result size limit
#     .config("spark.local.dir", "temp")  # Specify a directory with enough space
#     .config("spark.shuffle.file.buffer", "128k")  # Shuffle buffer size
#     .config("spark.memory.fraction", "0.8")  # Memory fraction for tasks
#     .config("spark.shuffle.memoryFraction", "0.6")  # Shuffle memory fraction
#     .config("spark.executor.javaOptions", "-Xmx48g")  # JVM heap size for executors
#     .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:1.0.8")
#     .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven")
#     .master("local[64]")  # Use all 64 cores on the machine
#     .getOrCreate()
# )

# spark

# spark = (
#     SparkSession
#     .builder
#     .appName("leash belka3")
#     .config("spark.driver.memory", "64g")  # Increased driver memory
#     .config("spark.executor.memory", "64g")  # Increased executor memory
#     .config("spark.executor.instances", "8")  # Reduced number of executors
#     .config("spark.executor.cores", "8")  # Increased cores per executor
#     .config("spark.driver.maxResultSize", "4g")  # Driver result size limit
#     .config("spark.local.dir", "temp")  # Specify a directory with enough space
#     .config("spark.shuffle.file.buffer", "128k")  # Shuffle buffer size
#     .config("spark.memory.fraction", "0.8")  # Memory fraction for tasks
#     .config("spark.shuffle.memoryFraction", "0.7")  # Shuffle memory fraction
#     .config("spark.executor.javaOptions", "-Xmx64g")  # JVM heap size for executors
#     .config("spark.sql.shuffle.partitions", "1000")  # Increase shuffle partitions
#     .config("spark.ui.enabled", "true")  # Enable Spark UI
#     .master("local[8]")  # Reduced number of cores for local mode
#     .getOrCreate()
# )

# spark


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/27 00:45:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/27 00:45:47 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [4]:
df0_features = spark.read.format('parquet').load('zero_features.parquet')
df1_features = spark.read.format('parquet').load('one_features.parquet')

full_df = df0_features.union(df1_features).orderBy(F.rand())

# print(df0_features.rdd.getNumPartitions())
# print(full_df.count())
# df0_features.printSchema()

In [5]:
# sample_df = full_df.sample(fraction=0.00001)

In [6]:
from pyspark.ml.feature import OneHotEncoder

protein_ohe = OneHotEncoder(inputCol="protein", outputCol="protein_onehot")
protein_ohe = protein_ohe.fit(full_df)

In [7]:
full_df = protein_ohe.transform(full_df)

In [8]:
features_cols = full_df.columns[-1:] + full_df.columns[2:-2]

In [9]:
vectorAssembler = VectorAssembler(inputCols=features_cols, outputCol='features')

In [10]:
full_df2 = vectorAssembler.transform(full_df)

In [11]:
# print(full_df2.rdd.getNumPartitions())

In [12]:
# full_df2 = full_df2.repartition(500)

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [13]:
import argparse
import os
import subprocess
import sys
from packaging import version

import numpy as np

import pyspark
import pyspark.sql.types as T
from pyspark import SparkConf
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import horovod.spark.torch as hvd
from horovod.spark.common.backend import SparkBackend
from horovod.spark.common.store import Store

Extension horovod.torch has not been built: /home/23m1521/.conda/envs/cuda_env2/lib/python3.12/site-packages/horovod/torch/mpi_lib_v2.cpython-312-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still available.


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

class BinaryClassifier(nn.Module):
    def __init__(self, input_dim):
        super(BinaryClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

cuda


In [15]:
train_df, test_df = full_df2.randomSplit([0.9, 0.1])

In [16]:
store = Store.create('/scratch/23m1521/temp2')

In [17]:
input_dim = 99
model = BinaryClassifier(input_dim).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
# Train a Horovod Spark Estimator on the DataFrame
backend = SparkBackend(num_proc=spark.sparkContext.defaultParallelism,
                       stdout=sys.stdout, stderr=sys.stderr,
                       prefix_output_with_timestamp=True)

torch_estimator = hvd.TorchEstimator(backend=backend,
                                     store=store,
                                     model=model,
                                     optimizer=optimizer,
                                     loss=lambda input, target: criterion(input, target.long()),
                                     input_shapes=[[-1, 99]],
                                     feature_cols=['features'],
                                     label_cols=['y'],
                                     batch_size=256,
                                     epochs=10,
                                     validation=0.1,
                                     backward_passes_per_step=1,
                                     verbose=1,
                                     use_gpu=True
                                    )
torch_estimator

TorchEstimator_6040fd634650

In [19]:
torch_model = torch_estimator.fit(train_df).setOutputCols(['prediction'])

num_partitions=640
writing dataframes
train_data_path=file:///scratch/23m1521/temp2/intermediate_train_data.0
val_data_path=file:///scratch/23m1521/temp2/intermediate_val_data.0


train_partitions=576


[Stage 12:>                                                      (0 + 64) / 200]

[571.352s][warning][gc,alloc] Executor task launch worker for task 13.0 in stage 12.0 (TID 549): Retried waiting for GCLocker too often allocating 4194304 words
[571.358s][warning][gc,alloc] Executor task launch worker for task 57.0 in stage 12.0 (TID 593): Retried waiting for GCLocker too often allocating 4194304 words
[571.358s][warning][gc,alloc] Executor task launch worker for task 33.0 in stage 12.0 (TID 569): Retried waiting for GCLocker too often allocating 4194304 words
[571.358s][warning][gc,alloc] Executor task launch worker for task 54.0 in stage 12.0 (TID 590): Retried waiting for GCLocker too often allocating 325781 words


24/12/27 00:55:02 WARN TaskMemoryManager: Failed to allocate a page (33554416 bytes), try again.
24/12/27 00:55:02 WARN TaskMemoryManager: Failed to allocate a page (33554416 bytes), try again.
24/12/27 00:55:02 WARN TaskMemoryManager: Failed to allocate a page (33554416 bytes), try again.
                                                                                

val_partitions=64


AttributeError: 'pyarrow.lib.Schema' object has no attribute 'to_arrow_schema'

In [ ]:
pred_df = torch_model.transform(test_df)

argmax = udf(lambda v: float(np.argmax(v)), returnType=T.DoubleType())
pred_df = pred_df.withColumn('label_pred', argmax(pred_df.label_prob))
evaluator = MulticlassClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderPR')
print('Test accuracy:', evaluator.evaluate(pred_df))

spark.stop()

24/12/27 02:03:40 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /scratch/23m1521/temp/blockmgr-b2a91130-8c98-4f76-a6f3-29c11d4eb0b1. Falling back to Java IO way
java.io.IOException: Failed to delete: /scratch/23m1521/temp/blockmgr-b2a91130-8c98-4f76-a6f3-29c11d4eb0b1
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:174)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:368)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1$adapted(DiskBlockManager.scala:364)
	at s

///////////////////////////////////////////////////////////////////////////////////////////////////////////////////